In [1]:
# Import libraries.
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import pandas as pd
import time
import random
from IPython.display import clear_output
from PIL import Image

%matplotlib inline

In [2]:
class GridEnvironment():
    
    def __init__(self, fin_x, fin_y, route_img):
        self.fin_x = fin_x
        self.fin_y = fin_y
        
        
        # RECOMPENSA
        self.rw = -1 # Living (Movement) Penalty
        
        # CARGA IMAGEN
        im = Image.open(route_img, 'r')
        
        # DIMENSIONES
        self.columns = im.size[0]
        self.rows = im.size[1]
        
        # INICIALIZACION
        pix_val = list(im.getdata())
        self.new_pix = [x[0] for x in pix_val]
        self.walls_and_paths = np.ones([self.rows,self.columns])
        for i in range(self.rows):
            for j in range(self.columns):
                if self.new_pix[i*self.columns+j] == 0:
                    self.walls_and_paths[i,j] = 0
        self.rewards = self.rw*np.ones([self.rows,self.columns])
        
        # Cambiar el valor
        self.value_state_table = np.zeros([self.rows,self.columns])
        self.value_state_table[fin_x][fin_y]=1000
    
    def getStateValue(self, position):
        return self.value_state_table[position[0]][position[1]]
    
    def getValue_state_table(self):
        return self.value_state_table
    
    def reset(self, fin_x, fin_y):
        self.value_state_table = np.zeros([self.rows,self.columns])
        self.value_state_table[fin_x][fin_y]=100

In [3]:
#the position array is created
array_pos=[]

class valueBasedAgent():
    
    def __init__(self, environment, policy, discount_factor):
        self.pos = [0,0]
        self.total_reward = 0
        self.environment = environment
        self.discount_factor = discount_factor
        self.actions = ["up", "down", "left", "right"]
        
        # Start with a random policy. 0.25 chance of moving to any direction.
        self.policy = policy   
            
    def forwardState(self, pos, action):
        
        # New position array.
        tam_y = len(environment.value_state_table[0])-1
        tam_x = len(environment.value_state_table)-1
        new_position = pos
        
        # Compute new position based on action taken.
        if(action == "up" and pos[1] < tam_y ):
            if(self.environment.walls_and_paths[pos[0]][pos[1] + 1]) == 1:
                new_position = [pos[0], pos[1] + 1]

        elif(action == "down" and pos[1] > 0):
            if(self.environment.walls_and_paths[pos[0]][pos[1] - 1]) == 1:
                new_position = [pos[0], pos[1] - 1]
                
        elif(action == "left" and pos[0] > 0):
            if(self.environment.walls_and_paths[pos[0] - 1][pos[1]]) == 1:
                new_position = [pos[0] - 1, pos[1]]

        elif(action == "right" and pos[0] < tam_x):
            if(self.environment.walls_and_paths[pos[0] + 1][pos[1]]) == 1:
                new_position = [pos[0] + 1, pos[1]]
        return new_position
        
        
    def valueFunction(self):
            
        # Initialize variable.
        new_state_value = 0
    
        # Random movement! - Cuando aun no se ha inicializado 
        if self.policy[self.pos[0]][self.pos[1]] == "r":
            for action in self.actions:        
                forward_state = self.forwardState(self.pos, action)
                
                # Simplified version of Q-value. BELLMANS EQUATION
                q_value = (self.environment.rewards[forward_state[0]][forward_state[1]] 
                                    + self.discount_factor * self.environment.value_state_table[forward_state[0]][forward_state[1]])
                new_state_value += q_value * 0.25 # Probabilidad de 0.25 para cada una de las acciones
            return new_state_value
        
        # Not random movement!
        else: 
            action = self.policy[self.pos[0]][self.pos[1]]
            forward_state = self.forwardState(self.pos, action)
            
            # Simplified version of Q-value.
            q_value = (self.environment.rewards[forward_state[0]][forward_state[1]] 
                                    + self.discount_factor * self.environment.value_state_table[forward_state[0]][forward_state[1]])
            new_state_value += q_value # Probabilidad de 1
            return new_state_value
        
    def getPosition(self):
        return self.pos
    
    def getReward(self):
        return self.total_reward
    
    def setPosition(self, x, y):
        self.pos = [x, y]
        
    def updateValueStateTable(self):
        new_state_value = self.valueFunction()
        self.environment.value_state_table[self.pos[0]][self.pos[1]] = new_state_value
        
    def selectBestAction(self):
        
        # Compute new possible states.
        go_up = self.forwardState(self.pos, "up")
        go_down = self.forwardState(self.pos, "down")
        go_left = self.forwardState(self.pos, "left")
        go_right = self.forwardState(self.pos, "right")
        
        # Q values (simplified version).
        up_value = (self.environment.rewards[go_up[0]][go_up[1]] + 
                    self.discount_factor * self.environment.value_state_table[go_up[0]][go_up[1]])
        down_value = (self.environment.rewards[go_down[0]][go_down[1]] + 
                      self.discount_factor * self.environment.value_state_table[go_down[0]][go_down[1]])
        left_value = (self.environment.rewards[go_left[0]][go_left[1]] + 
                        self.discount_factor * self.environment.value_state_table[go_left[0]][go_left[1]])
        right_value = (self.environment.rewards[go_right[0]][go_right[1]] + 
                       self.discount_factor * self.environment.value_state_table[go_right[0]][go_right[1]])
        
        # Array of Q-values.
        values = [up_value, down_value, left_value, right_value]
        
        best_action = self.actions[values.index(max(values))] 
        return best_action       
            
    def move(self):
    
        # Select action according to policy.
        action = self.policy[self.pos[0]][self.pos[1]]

        # Move to new position according to action taken.
        self.pos = self.forwardState(self.pos, action)
        

In [4]:
class PolicyEvaluation():#Graphics):
    
    def __init__(self,x_fin,y_fin, environment, agent, iterations = 3):
        
        self.x_fin = x_fin
        self.y_fin = y_fin
        self.environment = environment       
        self.agent = agent                     
        #print("GridWorld Initialize!")
        self.iterations = iterations
    
    def evaluate(self, plot_grid = True):
        self.DP_policy_evaluation(self.iterations, plot_grid)
        
    def DP_policy_evaluation(self, iterations, plot_grid):
        
        for k in range(0, iterations):
            print("Iteracion #",k)
            for i in range(0, len(self.environment.value_state_table)):
                for j in range(0, len(self.environment.value_state_table[0])):

                    if self.environment.walls_and_paths[i][j] == 1 and self.canChangeStateValue(i, j):

                        # Set agent position.
                        self.agent.setPosition(i, j)
                        self.agent.updateValueStateTable()
                            
    

    def canChangeStateValue(self, x, y):
        # Posicion que no se puede modificar
        cant_modify = bool((x == self.x_fin and y == self.y_fin)) # or (x == 4 and y == 4))
        
        grid = self.environment.walls_and_paths
        coords = list()
        
        # Get walls.
        for i in range(0, len(grid)):
            for j in range(0, len(grid[0])):
                if grid[i][j] == 0:
                    coords.append([i, j])
        for c in coords: 
            if c == [x, y]:
                cant_modify = True
                break
                
        return not cant_modify
    
    def updatePolicy(self):
        
         for i in range(0, len(self.environment.value_state_table)):
                for j in range(0, len(self.environment.value_state_table[0])):
                    if self.environment.walls_and_paths[i][j] == 1:
                        
                        # Set agent position.
                        self.agent.setPosition(i, j)
                        best_action = self.agent.selectBestAction()
                        self.agent.policy[i][j] = best_action

In [5]:

class Game():#Graphics):
    
    def __init__(self,x_fin,y_fin, environment, agent):
        
        self.environment = environment       
        self.agent = agent             
        print("GridWorld Initialize!")
                
    def update(self, secs):
        route = []
        pos = self.agent.getPosition()
        while not ((self.agent.pos[0] == x_fin and self.agent.pos[1] == y_fin)):
            
            self.agent.move()
            pos = self.agent.getPosition()
            print(pos)
            route.append(pos)
        #return route
            

### 1. Policy Evaluation (Planning) for DP.

In [18]:
# Generate the random policy.
policy = list()
for i in range(0, 70):
    column = list()
    for j in range(0, 80):
        column.append("r")
    policy.append(column)

# Initialize environment and agent.
x_fin = 13
y_fin = 3
discount_factor = 1
environment = GridEnvironment(x_fin,y_fin, "imagen1.png")
agent = valueBasedAgent(environment, np.array(policy), discount_factor)

agent.policy.shape

(70, 80)

In [19]:
environment.value_state_table.shape

(70, 80)

In [20]:
environment.walls_and_paths[12,2]

1.0

In [21]:
# Initialize policy evaluation class.
policy_evaluation = PolicyEvaluation(x_fin,y_fin,environment, agent, iterations = 15)
policy_evaluation.evaluate(plot_grid = False)

policy_evaluation.updatePolicy()

Iteracion # 0
Iteracion # 1
Iteracion # 2
Iteracion # 3
Iteracion # 4
Iteracion # 5
Iteracion # 6
Iteracion # 7
Iteracion # 8
Iteracion # 9
Iteracion # 10
Iteracion # 11
Iteracion # 12
Iteracion # 13
Iteracion # 14


In [22]:
# New agent policy after policy evaluation.
agent.policy

array([['d', 'd', 'd', ..., 'd', 'd', 'd'],
       ['l', 'd', 'l', ..., 'l', 'l', 'l'],
       ['l', 'd', 'd', ..., 'l', 'l', 'l'],
       ...,
       ['d', 'd', 'd', ..., 'd', 'd', 'd'],
       ['d', 'd', 'd', ..., 'd', 'd', 'd'],
       ['d', 'd', 'd', ..., 'd', 'd', 'd']], dtype='<U1')

In [23]:
politica = list()
for i in range(len(agent.policy)):
    row = list()
    for j in range(len(agent.policy[0])):
        if agent.policy[i,j] == 'u':
            row.append("up")
        if agent.policy[i,j] == 'd':
            row.append("down")
        if agent.policy[i,j] == 'r':
            row.append("right")
        if agent.policy[i,j] == 'l':
            row.append("left")
    politica.append(row)

In [24]:
politica =np.array(politica)
agent.policy = politica
agent.policy

array([['down', 'down', 'down', ..., 'down', 'down', 'down'],
       ['left', 'down', 'left', ..., 'left', 'left', 'left'],
       ['left', 'down', 'down', ..., 'left', 'left', 'left'],
       ...,
       ['down', 'down', 'down', ..., 'down', 'down', 'down'],
       ['down', 'down', 'down', ..., 'down', 'down', 'down'],
       ['down', 'down', 'down', ..., 'down', 'down', 'down']], dtype='<U5')

In [25]:
#environment.value_state_table

#### Win the Game with the previous policy evaluation results.

In [31]:
agent.pos = [14,6] #13, 3
array_pos.append (agent.pos)
game = Game(x_fin,y_fin, environment, agent)
game.update(0.3)

GridWorld Initialize!
[14, 5]
[14, 4]
[14, 3]
[13, 3]


In [ ]:
agent.policy

In [ ]:
print (array_pos)
#agent.pos = [4, 1]
#game = Game(x_fin,y_fin,environment, agent)
#game.update(0.1)

In [87]:
#agent.pos = [0, 0]
#game = Game(x_fin,y_fin,environment, agent)
#game.update(0.5)

In [88]:
# Generate the random policy.
policy = list()
for i in range(0, 5):
    column = list()
    for j in range(0, 5):
        column.append("r")
    policy.append(column)
    
# Initaliza environment and agent.
discount_factor = 0.5
environment = GridEnvironment(x_fin,y_fin)
agent = valueBasedAgent(environment, policy, discount_factor)

# Policy iteration algorithm.
for i in range(0, 1000):

    # Reset value function.
    environment.reset(x_fin,y_fin)

    # Evaluate new policy.
    policy_evaluation = PolicyEvaluation(x_fin,y_fin, environment, agent, iterations = 10)
    policy_evaluation.evaluate(plot_grid = False)
    policy_evaluation.updatePolicy()

#### The optimal policy.

In [89]:
agent.policy

[['up', 'right', 'down', 'down', 'down'],
 ['r', 'right', 'r', 'r', 'left'],
 ['up', 'up', 'up', 'up', 'r'],
 ['left', 'r', 'up', 'left', 'down'],
 ['left', 'down', 'r', 'left', 'down']]

In [90]:
# Generate the random policy.
policy = list()
for i in range(0, 5):
    column = list()
    for j in range(0, 5):
        column.append("r")
    policy.append(column)
    
# Initaliza environment and agent.
discount_factor = 0.6
environment = GridEnvironment(x_fin,y_fin)
agent = valueBasedAgent(environment, policy, discount_factor)

# Policy iteration algorithm.
for i in range(0, 1000):

    # Reset value function.
    # environment.reset() => We do not reset the environment? 

    # Evaluate new policy.
    policy_evaluation = PolicyEvaluation(x_fin,y_fin,environment, agent, iterations = 1)
    policy_evaluation.evaluate(plot_grid = False)
    policy_evaluation.updatePolicy()

#### The optimal policy.

In [91]:
agent.policy

[['up', 'right', 'down', 'down', 'down'],
 ['r', 'right', 'r', 'r', 'left'],
 ['up', 'up', 'up', 'up', 'r'],
 ['left', 'r', 'up', 'left', 'down'],
 ['left', 'down', 'r', 'left', 'down']]

In [92]:
#function that debugs the position array and returns only the desired coordinates
def coordenates (route:list):
    final_coordenates=[]
    
    for i in range (1,len (route) -1):
        if route [i][0] == route [i-1][0] :
            if route [i][0] != route [i+1][0] :
                final_coordenates.append (route[i])
        elif route [i][1] == route [i-1][1] :
            if route [i][1] != route [i+1][1] :
                final_coordenates.append (route[i])                      
    final_coordenates.append (route[len (route)-1])        
    return final_coordenates
coordenates (array_pos)



[[4, 0], [2, 0], [2, 3]]

In [93]:
#the array is reinitialized
array_pos=[]